# Clustering Crypto

In [34]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [35]:
# Load the crypto_data.csv dataset.
full_crypto_df = pd.read_csv("Resources/crypto_data.csv", index_col=0)
full_crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [36]:
# Keep all the cryptocurrencies that are being traded.
traded_crypto_df = full_crypto_df[full_crypto_df["IsTrading"] == True]
traded_crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [37]:
# Keep all the cryptocurrencies that have a working algorithm.
working_crypto_df = traded_crypto_df[traded_crypto_df["Algorithm"] != "NaN"]
working_crypto_df.head(5)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [38]:
# Remove the "IsTrading" column. 
working_crypto_df = working_crypto_df[['CoinName', 'Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply']]
working_crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [39]:
# Remove rows that have at least 1 null value.
working_crypto_df.notnull().sum()

cleaned_crypto_df = working_crypto_df.dropna() 
cleaned_crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [40]:
# Keep the rows where coins are mined.
cleaned_crypto_df.drop(cleaned_crypto_df[cleaned_crypto_df['TotalCoinsMined'] ==0].index, inplace=True)
cleaned_crypto_df.head(10)

#cleaned_crypto_df.notnull().sum()

C:\Users\auror\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [41]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = cleaned_crypto_df[["CoinName"]]
names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [42]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = cleaned_crypto_df.drop(columns ="CoinName", axis = 1)
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [43]:
# Use get_dummies() to create variables for text features.
x = pd.get_dummies(crypto_df, columns =["Algorithm", "ProofType"])
x.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# Standardize the data with StandardScaler().
x_scaled = StandardScaler().fit_transform(x)
print(x_scaled[0:5])

[[-0.11674788 -0.15286468 -0.0433555  -0.0433555  -0.0433555  -0.06137164
  -0.07523548 -0.0433555  -0.06137164 -0.06137164 -0.0433555  -0.0433555
  -0.19226279 -0.06137164 -0.09731237 -0.0433555  -0.11536024 -0.07523548
  -0.0433555  -0.0433555  -0.15176505 -0.0433555  -0.13105561 -0.0433555
  -0.0433555  -0.08695652 -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.06137164 -0.0433555  -0.08695652 -0.08695652 -0.08695652 -0.0433555
  -0.13105561 -0.13827675 -0.13827675 -0.0433555  -0.06137164 -0.0433555
  -0.07523548 -0.1815096  -0.0433555  -0.0433555  -0.0433555  -0.07523548
  -0.15811388 -0.3145935  -0.0433555  -0.08695652 -0.07523548 -0.06137164
  -0.0433555   1.38873015 -0.0433555  -0.0433555  -0.06137164 -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.0433555  -0.39836623 -0.0433555  -0.1815096  -0.0433555  -0.08695652
  -0.08695652 -0.10670145 -0.0433555  -0.0433555  -0.13105561 -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.07523

### Deliverable 2: Reducing Data Dimensions Using PCA

In [45]:
# Using PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
x_pca = pca.fit_transform(x_scaled)
x_pca

array([[-0.32636299,  0.99902356, -0.48025886],
       [-0.30967773,  0.99911991, -0.48043105],
       [ 2.30799529,  1.67509555, -0.59346947],
       ...,
       [ 0.3276684 , -2.30517753,  0.35397007],
       [-0.13492886, -1.97293337,  0.36258917],
       [-0.27529627,  0.81579396, -0.23912338]])

In [46]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data=x_pca, columns=["pc1", "pc2", "pc3"], index=crypto_df.index)
pcs_df

,pc1,pc2,pc3
42,-0.326363,0.999024,-0.480259
404,-0.309678,0.999120,-0.480431
1337,2.307995,1.675096,-0.593469
BTC,-0.144632,-1.264582,0.145922
ETH,-0.136677,-1.985697,0.351867
...,...,...,...
ZEPH,2.470984,0.995834,-0.118445
GAP,-0.324406,0.998891,-0.480260
BDX,0.327668,-2.305178,0.353970
ZEN,-0.134929,-1.972933,0.362589


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [47]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\auror\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [55]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions


array([1, 1, 1, 3, 3, 3, 1, 3, 3, 3, 1, 3, 1, 1, 3, 1, 3, 3, 1, 1, 3, 3,
       3, 3, 3, 1, 3, 3, 3, 1, 3, 1, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 1, 1,
       3, 3, 3, 3, 3, 1, 1, 3, 1, 3, 3, 3, 3, 1, 3, 3, 1, 3, 1, 1, 1, 3,
       3, 3, 1, 1, 1, 1, 1, 3, 3, 3, 1, 1, 3, 1, 3, 1, 1, 3, 3, 3, 3, 1,
       1, 3, 1, 3, 3, 1, 1, 3, 1, 1, 3, 3, 3, 1, 3, 1, 1, 3, 1, 3, 1, 3,
       1, 3, 1, 1, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 1,
       3, 1, 3, 3, 1, 3, 1, 3, 1, 1, 3, 3, 1, 3, 3, 1, 1, 3, 1, 3, 1, 1,
       1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 3, 1, 3, 1, 1, 3, 1, 3, 1, 1, 3, 1, 3, 1, 3, 1, 3, 1, 1, 1, 1,
       3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 3, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 1, 1, 1, 3, 1, 3, 1,
       1, 3, 1, 3, 3, 1, 3, 3, 1, 3, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 3, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 3, 3, 3, 3, 1, 3, 1, 1, 3,
       1, 3, 3, 3, 1, 3, 1, 3, 3, 3, 1, 3, 1, 3, 1,

In [56]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis =1, join='outer')

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = names_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = model.labels_
clustered_df

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,pc1,pc2,pc3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.326363,0.999024,-0.480259,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.309678,0.999120,-0.480431,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.307995,1.675096,-0.593469,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144632,-1.264582,0.145922,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.136677,-1.985697,0.351867,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.152972,-1.143168,0.023131,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.405881,1.228302,-0.471162,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.161053,-2.229708,0.406596,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.135117,-1.985811,0.351867,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.134928,-1.972933,0.362589,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [60]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="pc1",
    y="pc2",
    z="pc3",
    color="Class",
    symbol="Class",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [61]:
# Create a table with tradable cryptocurrencies.

clustered_df[["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"]].hvplot.table()

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [62]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {len(clustered_df)} tradable cryptocurrencies")

There are 533 tradable cryptocurrencies


In [72]:
# Scaling data to create the scatter plot with tradable cryptocurrencies w 
#MinMaxScaler on Total Coin Supply and TotalCoinsMined

to_scale_df = clustered_df[["TotalCoinSupply","TotalCoinsMined"]]
scaled = MinMaxScaler().fit_transform(to_scale_df) 
scaled

array([[4.20000000e-11, 5.94230127e-03],
       [5.32000000e-04, 7.00182308e-03],
       [3.14159265e-01, 3.53420682e-02],
       ...,
       [1.40022261e-03, 6.92655266e-03],
       [2.10000000e-05, 5.94962775e-03],
       [1.00000000e-06, 5.94243008e-03]])

In [82]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_df = pd.DataFrame(data=scaled, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_df["CoinName"]=names_df

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_df["Class"]=model.labels_
scaled_df

#plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.005942,42 Coin,1
404,5.320000e-04,0.007002,404Coin,1
1337,3.141593e-01,0.035342,EliteCoin,1
BTC,2.100000e-05,0.005960,Bitcoin,3
ETH,0.000000e+00,0.006050,Ethereum,3
...,...,...,...,...
ZEPH,2.000000e-03,0.007951,ZEPHYR,1
GAP,2.500000e-04,0.005957,Gapcoin,1
BDX,1.400223e-03,0.006927,Beldex,3
ZEN,2.100000e-05,0.005950,Horizen,3


In [84]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class"
)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)